In [1]:
from torch.utils.data import TensorDataset
from transformers import DistilBertForSequenceClassification, AdamW
from transformers import DistilBertModel, DistilBertTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import numpy as np
import pandas as pd
import pickle
import torch


/home/roland/Projects/vananh/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#!pip install torch torchvision torchaudio


In [3]:
sentences_clean = pd.read_pickle("../resources/sentences_clean.pkl")

In [2]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load pre-trained model (weights)
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [4]:
# Batch tokenization
def batch_tokenize(sentences, batch_size=100):  # adjust the batch_size based memory constraints
    input_ids, attention_masks = [], []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        tokenized = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids.append(tokenized['input_ids'])
        attention_masks.append(tokenized['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

In [5]:
# Tokenize the entire dataset in batches
input_ids, attention_mask = batch_tokenize(sentences_clean['sentence'].tolist())

KeyboardInterrupt: 

In [27]:
# Set device
#device = "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Move the model to the device
model = model.to(device)

# Function to process data in smaller chunks
def process_in_chunks(model, input_ids, attention_mask, chunk_size=32):
    model.eval()  # Evaluation mode
    total_chunks = len(input_ids) // chunk_size + (0 if len(input_ids) % chunk_size == 0 else 1)
    all_embeddings = []

    for i in range(total_chunks):
        start_idx = i * chunk_size
        end_idx = start_idx + chunk_size

        # Slicing the tensors and moving them to the same device as the model
        input_ids_chunk = input_ids[start_idx:end_idx].to(device)
        attention_mask_chunk = attention_mask[start_idx:end_idx].to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids_chunk, attention_mask=attention_mask_chunk)
            embeddings_chunk = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeddings.append(embeddings_chunk)

    return np.concatenate(all_embeddings, axis=0)

# Call the function with tensors moved to the same device as the model
embeddings = process_in_chunks(model, input_ids.to(device), attention_mask.to(device))  


In [17]:
"""
# Ensure the model is in evaluation mode
model.eval()

# Move tensors to the same device as the model (GPU or CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
model = model.to(device)

# Generate embeddings
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
"""

RuntimeError: [enforce fail at alloc_cpu.cpp:83] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 22878346543104 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
embeddings

In [ ]:
"""evalluate model with validation data"""

In [ ]:
np.save('../resources/sentence_embeddings.npy', embeddings)